In [23]:
### Settings

import requests, json, time, datetime, pytz, pandas as pd, cv2, multiprocessing 
from IPython.display import clear_output as co
from google.cloud import storage
from google.oauth2 import service_account
from apscheduler.schedulers.background import BackgroundScheduler, BlockingScheduler

# Google storage client and bucket
credentials = service_account.Credentials.from_service_account_file('../../../../Apps/Python/bolsao-api/credentials/pluvia-360323-eae2907a9c98.json')
gcs_client = storage.Client(credentials=credentials)
bucket = gcs_client.get_bucket('city-camera-images')

def create_folder(folder):
    blob = bucket.blob(f'{folder}/')
    blob.upload_from_string('', content_type='application/x-www-form-urlencoded;charset=UTF-8')

cams_url = 'http://187.111.99.18:9004/?CODE={}'
google_storage_url = '/datasets/city-camera-data-collection'  # 100GB -> R$20,00 / mês

### Recorder class

class recorder:

    cams_url = 'http://187.111.99.18:9004/?CODE={}'
    google_storage_url = '/datasets/city-camera-data-collection'  # 100GB -> R$20,00 / mês

    def __init__(self, max_frames, workers=4):
        self.max_frames = max_frames
        self.workers = workers
        self.timegroup = datetime.datetime.now(pytz.timezone('Brazil/East')).strftime('%Y-%m-%d %X')

    def record(self, code, folder):
        path = f'{folder}/{code}/{self.timegroup}'
        create_folder(path)
        cap = cv2.VideoCapture(self.cams_url.format(code))
        i = -1; print(f'Start for cam {code}')
        while(True):
            i += 1
            if i == self.max_frames: break
            success, frame = cap.read()
            while not success:
                cap = cv2.VideoCapture(self.cams_url.format(code))
                success, frame = cap.read()
            timestamp = datetime.datetime.now(pytz.timezone('Brazil/East')).strftime('%Y-%m-%d %X')
            filename = f'CODE{code}_{timestamp}'
            cv2.imwrite(f'{self.google_storage_url}/{path}/{filename}.jpg', frame)

            co(1); print(f'Camera {code} images saved: {i}.')

    def record_in_parallel(self, codes, folder):
        now = datetime.datetime.now(pytz.timezone('Brazil/East'))
        self.timegroup = now.strftime('%Y-%m-%d %X')
        workers = len(codes) if self.workers == 'auto' else self.workers
        items = [(code, folder) for code in codes]
        pool = multiprocessing.Pool(processes=workers)
        pool.starmap(self.record, items)
        return



### Request data

api_root = 'https://bolsao-api-j2fefywbia-rj.a.run.app'

def endpoint_df(endpoint):
    return pd.DataFrame(requests.get(f'{api_root}/{endpoint}').json())
    
polygons = endpoint_df('mongo/gbf/Polygons')
polygons_alertario = endpoint_df('mongo/Polygons AlertaRio')
cameras = endpoint_df('mongo/Cameras AlertaRio')

poly_cams = cameras[cameras['cluster_id'] != -1] # cameras inside polygons

poly_cams_codes = poly_cams['Codigo'].astype(str).tolist()

### Schedule and record images

In [ ]:
river_codes = ['3323', '1487', '3318']
otavio = ['3104', '3127']

folder = 'pics'
codes =  otavio + river_codes + poly_cams_codes

rec = recorder(max_frames=10, workers='auto')

# ---
# Scheduler object and jobs

sched = BlockingScheduler()

@sched.scheduled_job('cron', minute='*/5')
def collect_cams_imgs():
    rec.record_in_parallel(codes, folder)
    co(1); print('Image group {rec.timegroup} finished.')
    return

sched.start()

### Load live images

In [7]:
def record(code):
    cap = cv2.VideoCapture(api_url+code)
    i = 0
    print('Starting')
    while(True):
        co(1); print(f'Image {i} saved.')
        i += 1
        if i==10: break
        success, frame = cap.read()
        while not success:
            cap = cv2.VideoCapture(api_url+code)
            success, frame = cap.read()

        curr_time = datetime.datetime.now(pytz.timezone('Brazil/East')).strftime('%Y-%m-%d %X')
        filename = f'CODE{code}_{curr_time}'
#         cv2.imwrite(f'{foldername}/{filename}.jpg', frame)
        cv2.imwrite(f'{filename}.jpg', frame)

api_url = 'http://187.111.99.18:9004/?CODE='
codes = ['313']
foldername = 'images'

def main(codes):
    items = [(code) for code in codes]
    pool = multiprocessing.Pool(processes=1)
    pool.starmap(record, items)
    return

In [8]:
record('313')

Starting
Image 0 saved.
Image 1 saved.
Image 2 saved.
Image 3 saved.
Image 4 saved.
Image 5 saved.
Image 6 saved.
Image 7 saved.
Image 8 saved.
Image 9 saved.


In [ ]:
main(codes)

In [12]:
cap = cv2.VideoCapture(api_url+'313')
success, frame = cap.read()
cv2.imwrite(f'images/test.jpg', frame)

frame

array([[[ 94,  79,  64],
        [103,  88,  73],
        [ 65,  69,  67],
        ...,
        [115, 131, 131],
        [156, 172, 172],
        [217, 233, 233]],

       [[101,  86,  71],
        [ 85,  70,  55],
        [ 81,  85,  83],
        ...,
        [128, 144, 144],
        [154, 170, 170],
        [196, 212, 212]],

       [[ 82,  85,  74],
        [ 68,  71,  60],
        [ 79,  79,  83],
        ...,
        [130, 146, 146],
        [158, 174, 174],
        [192, 208, 208]],

       ...,

       [[ 39,  78, 110],
        [ 40,  79, 111],
        [ 42,  81, 113],
        ...,
        [ 60,  58,  59],
        [ 60,  58,  59],
        [ 60,  58,  59]],

       [[ 39,  78, 110],
        [ 39,  78, 110],
        [ 41,  80, 112],
        ...,
        [ 61,  59,  60],
        [ 61,  59,  60],
        [ 61,  59,  60]],

       [[ 39,  78, 110],
        [ 39,  78, 110],
        [ 41,  80, 112],
        ...,
        [ 61,  59,  60],
        [ 61,  59,  60],
        [ 61,  59,  60]]